# Classifying Playing Cards

In [1]:
import numpy as np
import os 
import PIL
import PIL.Image 
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt

## Importing the data from directory 

In [2]:
local_data_dir = 'D:/card_data/cv_num_cards_deck/52kards'

In [3]:
img_height = 224
img_width = 224
img_size = (224, 224)
batch_size = 32

In [4]:
# Create the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    local_data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width), # specified the resizing 
    batch_size=batch_size) 

Found 4376 files belonging to 52 classes.
Using 3501 files for training.


In [5]:
# Create the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    local_data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 4376 files belonging to 52 classes.
Using 875 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['10c', '10d', '10h', '10s', '2c', '2d', '2h', '2s', '3c', '3d', '3h', '3s', '4c', '4d', '4h', '4s', '5c', '5d', '5h', '5s', '6c', '6d', '6h', '6s', '7c', '7d', '7h', '7s', '8c', '8d', '8h', '8s', '9c', '9d', '9h', '9s', 'Ac', 'Ad', 'Ah', 'As', 'Jc', 'Jd', 'Jh', 'Js', 'Kc', 'Kd', 'Kh', 'Ks', 'Qc', 'Qd', 'Qh', 'Qs']


In [7]:
# visualize the first 9 images from the dataset
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [8]:
# image preprocessing 
normalization_layer = tf.keras.layers.Rescaling(1./127.5, offset=-1)
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
augmented_ds = normalized_ds # .map(lambda x, y: (data_augmentation(normalization_layer(x)), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
# naming hasn't changed because of laziness 

In [9]:
# need comments
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Training our own Neural Network (not appropriate for classification with more classes - defunct)
We're first going to try training our own convolutional neural network from scratch 


In [10]:
num_classes = 52

model_1 = tf.keras.Sequential([ # rescaling layer 
  tf.keras.layers.Conv2D(32, 3, activation='relu'), # relu activation function
  tf.keras.layers.Conv2D(32, 3, activation='relu'), # convolution layer  
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [11]:
model_1.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

In [12]:
# model_1.fit(
#   normalized_ds, # augmented_ds
#   validation_data=val_ds,
#   epochs=10
# )

## Assessing predictions from our model

In [13]:
def image_prediction(image_path, model):
    image = PIL.Image.open(image_path).resize((img_width, img_height))
    image_array = np.array(image) / 127.5 - 1  # Scale pixel values to -1 to 1
    image_array = np.expand_dims(image_array, axis=0)  # Add a batch dimension
    predictions = model.predict(image_array)
    probabilities = tf.nn.softmax(predictions[0])  # Apply softmax to convert logits to probabilities
    predicted_class = np.argmax(probabilities)  # Get the index of the highest probability
    print(f"Predicted class: {predicted_class}, Probability: {probabilities[predicted_class]}")

In [14]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/2c/IMG_E0924.JPG", model_1)

1/1 [==============================] - 1s 532ms/step
Predicted class: 2, Probability: 0.01927671954035759


In [15]:
class_names[7]

'2s'

In [16]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/6h/IMG_E0925.JPG", model_1)

1/1 [==============================] - 0s 51ms/step
Predicted class: 2, Probability: 0.019275179132819176


In [17]:
class_names[26]

'7h'

Our model is assessing all three classes as the same class with very similar probability.

# Model 2 

In [18]:
def image_prediction(image_path, model, class_names, img_width=224, img_height=224):
    image = PIL.Image.open(image_path).resize((img_width, img_height)).convert('RGB')
    image_array = np.array(image) / 127.5 - 1  # Scale pixel values to -1 to 1
    image_array = np.expand_dims(image_array, axis=0)  # Add a batch dimension

    predictions = model.predict(image_array)
    
    # Since the model's final layer uses a softmax activation, predictions are already in probability form
    predicted_class = np.argmax(predictions[0])  # Get the index of the highest probability class
    probability = predictions[0][predicted_class]  # Get the probability of the predicted class
    
    print(f"Predicted class: {predicted_class}, Probability: {probability}, Class: {class_names[predicted_class]}")
    return predicted_class, probability

In [19]:
epochs=20

In [20]:
# model_2 = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(52, activation='softmax')
# ])

In [21]:
# model_2 = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)), # change to input shape changed training time 
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(52, activation='softmax')
# ])

In [22]:
model_2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)), # change to input shape changed training time 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(52, activation='softmax')
])

In [23]:
model_2.compile(loss = 'sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [24]:
model_2.fit(
  normalized_ds, # augmented_ds
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
110/110 [==============================] - 217s 2s/step - loss: 3.7147 - accuracy: 0.1151 - val_loss: 2.2596 - val_accuracy: 0.2926
Epoch 2/10
110/110 [==============================] - 216s 2s/step - loss: 1.3056 - accuracy: 0.5767 - val_loss: 1.4735 - val_accuracy: 0.5669
Epoch 3/10
110/110 [==============================] - 212s 2s/step - loss: 0.4242 - accuracy: 0.8695 - val_loss: 2.5038 - val_accuracy: 0.5063
Epoch 4/10
110/110 [==============================] - 211s 2s/step - loss: 0.1819 - accuracy: 0.9512 - val_loss: 1.5269 - val_accuracy: 0.6160
Epoch 5/10
110/110 [==============================] - 207s 2s/step - loss: 0.1128 - accuracy: 0.9677 - val_loss: 1.9431 - val_accuracy: 0.6606
Epoch 6/10
110/110 [==============================] - 195s 2s/step - loss: 0.0809 - accuracy: 0.9803 - val_loss: 1.9365 - val_accuracy: 0.6389
Epoch 7/10
110/110 [==============================] - 197s 2s/step - loss: 0.0501 - accuracy: 0.9877 - val_loss: 2.0696 - val_accuracy: 0.6743

In [25]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/2c/IMG_E0924.JPG", model_2, class_names)

1/1 [==============================] - 0s 80ms/step
Predicted class: 4, Probability: 0.7655242085456848, Class: 2c


(4, 0.7655242)

In [26]:
class_names[4]

'2c'

In [27]:
# image = PIL.Image.open(image_path).resize((img_width, img_height))

In [28]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/6h/IMG_E0925.JPG", model_2, class_names)

1/1 [==============================] - 0s 43ms/step
Predicted class: 22, Probability: 0.9988002777099609, Class: 6h


(22, 0.9988003)

In [29]:
class_names[22]

'6h'

Alistair's Cards

In [30]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/6d/Image20240325150136.PNG", model_2, class_names)

1/1 [==============================] - 0s 43ms/step
Predicted class: 22, Probability: 1.0, Class: 6h


(22, 1.0)

In [31]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/Qh/Image20240325150115.PNG", model_2, class_names)

1/1 [==============================] - 0s 43ms/step
Predicted class: 50, Probability: 0.998157799243927, Class: Qh


(50, 0.9981578)

In [32]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/6d/Image20240325150136.PNG", model_2, class_names)

1/1 [==============================] - 0s 43ms/step
Predicted class: 22, Probability: 1.0, Class: 6h


(22, 1.0)

In [33]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/Ad/Image20240325150935.PNG", model_2, class_names)

1/1 [==============================] - 0s 43ms/step
Predicted class: 36, Probability: 1.0, Class: Ac


(36, 1.0)

In [34]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/4h/Image20240325150932.PNG", model_2, class_names)

1/1 [==============================] - 0s 44ms/step
Predicted class: 13, Probability: 0.9869118928909302, Class: 4d


(13, 0.9869119)

In [35]:
image_prediction("D:/card_data/cv_num_cards_deck/test_images/9s/Image20240325150930.PNG", model_2, class_names)

1/1 [==============================] - 0s 43ms/step
Predicted class: 24, Probability: 0.998310923576355, Class: 7c


(24, 0.9983109)